In [1]:
#https://docs.langchain.com/oss/python/langchain/sql-agent
import os

In [3]:
#this one works
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        google_api_key=os.environ["GEMINI_API_KEY"]
    )

response = llm.invoke("please list the country name order by land size in descending order.")
print("✅ LangChain-Gemini integration working correctly")
print(f"Response: {response.content}")

✅ LangChain-Gemini integration working correctly
Response: Here is a list of countries ordered by land area in descending order (largest to smallest). Please note that exact figures can vary slightly depending on the source and how inland waters or disputed territories are counted, but the general order remains consistent.

1.  **Russia**
2.  **China**
3.  **United States**
4.  **Canada**
5.  **Brazil**
6.  **Australia**
7.  **India**
8.  **Argentina**
9.  **Kazakhstan**
10. **Algeria**
11. **Democratic Republic of the Congo**
12. **Saudi Arabia**
13. **Mexico**
14. **Indonesia**
15. **Sudan**


In [7]:
import sqlite3
import pandas as pd

with sqlite3.connect(r'D:\data science data\ai_sqlite.db') as conn:    
        
    cursor = conn.cursor()
    sql_txt_create = '''                
    
                -- Enable foreign key constraints (important for data integrity)
                PRAGMA foreign_keys = ON;

                DROP TABLE IF EXISTS OrderDetails;

                -- Create the Products table 
                DROP TABLE IF EXISTS Products;               
                CREATE TABLE IF NOT EXISTS Products (
                    ProductID INTEGER PRIMARY KEY,
                    ProductName TEXT NOT NULL,
                    UnitPrice REAL NOT NULL
                );

                -- Create the Orders table
                DROP TABLE IF EXISTS Orders;                
                CREATE TABLE IF NOT EXISTS Orders (
                    OrderID INTEGER PRIMARY KEY,
                    OrderDate TEXT NOT NULL,
                    CustomerID INTEGER NOT NULL
                );

                -- Create the OrderDetails (junction) table to link Orders and Products
                DROP TABLE IF EXISTS OrderDetails;
                CREATE TABLE IF NOT EXISTS OrderDetails (
                    OrderDetailID INTEGER PRIMARY KEY,
                    OrderID INTEGER NOT NULL,
                    ProductID INTEGER NOT NULL,
                    Quantity INTEGER NOT NULL,
                    FOREIGN KEY (OrderID) REFERENCES Orders(OrderID) ON DELETE CASCADE,
                    FOREIGN KEY (ProductID) REFERENCES Products(ProductID) ON DELETE CASCADE
                );

                -- Insert multiple records into Products
                INSERT INTO Products (ProductName, UnitPrice) VALUES
                ('Laptop', 1200.00),
                ('Mouse', 25.50),
                ('Keyboard', 49.99),
                ('Monitor', 350.00);

                -- Insert multiple records into Orders
                INSERT INTO Orders (OrderDate, CustomerID) VALUES
                ('2025-12-01', 101),
                ('2025-12-05', 102),
                ('2025-12-10', 101);

                -- Insert multiple records into OrderDetails
                INSERT INTO OrderDetails (OrderID, ProductID, Quantity) VALUES
                (1, 1, 1), -- Order 1: 1 Laptop
                (1, 2, 2), -- Order 1: 2 Mouses
                (2, 3, 1), -- Order 2: 1 Keyboard
                (3, 1, 1), -- Order 3: 1 Laptop
                (3, 4, 2); -- Order 3: 2 Monitors

    '''
    # Execute the entire script at once
    cursor.executescript(sql_txt_create)    
    cursor.close()

    #query db to get data into dataframe
    #query to join mutliple table to get summary
    sql_txt_query  = '''
            SELECT
            O.OrderID,
            O.OrderDate,
            P.ProductName,
            OD.Quantity,
            P.UnitPrice,
            (OD.Quantity * P.UnitPrice) AS TotalPerItem
            FROM
                Orders O
            INNER JOIN
                OrderDetails OD ON O.OrderID = OD.OrderID
            INNER JOIN
                Products P ON OD.ProductID = P.ProductID
            ORDER BY
                O.OrderID, P.ProductName;

    '''
    #cursor.execute(sql_txt_query)
    df = pd.read_sql_query(sql_txt_query, conn)
df


,OrderID,OrderDate,ProductName,Quantity,UnitPrice,TotalPerItem
0,1,2025-12-01,Laptop,1,1200.00,1200.00
1,1,2025-12-01,Mouse,2,25.50,51.00
2,2,2025-12-05,Keyboard,1,49.99,49.99
3,3,2025-12-10,Laptop,1,1200.00,1200.00
4,3,2025-12-10,Monitor,2,350.00,700.00


In [ ]:
#https://docs.langchain.com/oss/python/langchain/sql-agent
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri(r"sqlite:///D:/data science data/ai_sqlite.db")
print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")
print(f'Sample output: {db.run("SELECT * FROM Products LIMIT 5;")}')

Dialect: sqlite
Available tables: ['OrderDetails', 'Orders', 'Products']
Sample output: [(1, 'Laptop', 1200.0), (2, 'Mouse', 25.5), (3, 'Keyboard', 49.99), (4, 'Monitor', 350.0)]


In [9]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for tool in tools:
    print(f"{tool.name}: {tool.description}\n")

sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



In [10]:
#Use create_agent
system_prompt = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
    dialect=db.dialect,
    top_k=10,
)

In [11]:
from langchain.agents import create_agent
agent = create_agent(
    llm,
    tools,
    system_prompt=system_prompt,
)

In [12]:
question = "what product is most expensive?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

what product is most expensive?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (44bb9e61-359a-40bc-ad16-8eff7832fbc5)
 Call ID: 44bb9e61-359a-40bc-ad16-8eff7832fbc5
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

OrderDetails, Orders, Products
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (18899686-ad88-4137-a861-2ccadaaa6154)
 Call ID: 18899686-ad88-4137-a861-2ccadaaa6154
  Args:
    table_names: Products
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Products" (
	"ProductID" INTEGER, 
	"ProductName" TEXT NOT NULL, 
	"UnitPrice" REAL NOT NULL, 
	PRIMARY KEY ("ProductID")
)

/*
3 rows from Products table:
ProductID	ProductName	UnitPrice
1	Lapt

In [13]:
#give a question that need to join 2 tables

question = "Which product has been ordered least by count number?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which product has been ordered least by count number?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (3e4790b0-4651-4e70-88c8-3563ca3ee42e)
 Call ID: 3e4790b0-4651-4e70-88c8-3563ca3ee42e
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

OrderDetails, Orders, Products
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (c9771228-c0ab-4050-a663-04bf8faaa8a6)
 Call ID: c9771228-c0ab-4050-a663-04bf8faaa8a6
  Args:
    table_names: Products, OrderDetails
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "OrderDetails" (
	"OrderDetailID" INTEGER, 
	"OrderID" INTEGER NOT NULL, 
	"ProductID" INTEGER NOT NULL, 
	"Quantity" INTEGER NOT NULL, 
	PRIMARY KEY ("OrderDe

In [14]:
#check if agent can handle more involved situation

question = "Which product has been ordered at 2nd place by sale amounts"  #this should be monitor $700, which is second after laptop

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which product has been ordered at 2nd place by sale amounts
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (15d3c6da-2b3c-411d-8fec-dc7229ed5216)
 Call ID: 15d3c6da-2b3c-411d-8fec-dc7229ed5216
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

OrderDetails, Orders, Products
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (ae1aa2c4-0660-4145-bf5d-5f1d88892c6c)
 Call ID: ae1aa2c4-0660-4145-bf5d-5f1d88892c6c
  Args:
    table_names: OrderDetails, Orders, Products
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "OrderDetails" (
	"OrderDetailID" INTEGER, 
	"OrderID" INTEGER NOT NULL, 
	"ProductID" INTEGER NOT NULL, 
	"Quantity" INTEGER NOT NULL, 
	PRIMARY

In [15]:
#This has been good, but in real life, many transaction database did not give you straightforward column name, use abbre a lot and  sometimes even obfuscate column name deliberately to protect data, want to see if langchain can handle that
#usually secondary database like data warehouse is more user friendly
#let me create a db with gibberish to see langchain recognize or not
with sqlite3.connect(r'D:\data science data\ai_sqlite2.db') as conn:    
        
    cursor = conn.cursor()
    sql_txt_create = '''                
    
                -- Enable foreign key constraints (important for data integrity)
                PRAGMA foreign_keys = ON;

                DROP TABLE IF EXISTS DingdanMingxi;

                -- Create the Chanping table 
                DROP TABLE IF EXISTS Chanping;               
                CREATE TABLE IF NOT EXISTS Chanping (
                    ChanpingID INTEGER PRIMARY KEY,
                    ChanpingName TEXT NOT NULL,
                    Danjia REAL NOT NULL
                );

                -- Create the Dingdan table
                DROP TABLE IF EXISTS Dingdan;                
                CREATE TABLE IF NOT EXISTS Dingdan (
                    DingdanID INTEGER PRIMARY KEY,
                    DingdanRiqi TEXT NOT NULL,
                    GukeID INTEGER NOT NULL
                );

                -- Create the DingdanMingxi (junction) table to link Dingdan and Chanping
                DROP TABLE IF EXISTS DingdanMingxi;
                CREATE TABLE IF NOT EXISTS DingdanMingxi (
                    DingdanMingxiID INTEGER PRIMARY KEY,
                    DingdanID INTEGER NOT NULL,
                    ChanpingID INTEGER NOT NULL,
                    Shuliang INTEGER NOT NULL,
                    FOREIGN KEY (DingdanID) REFERENCES Dingdan(DingdanID) ON DELETE CASCADE,
                    FOREIGN KEY (ChanpingID) REFERENCES Chanping(ChanpingID) ON DELETE CASCADE
                );

                -- Insert multiple records into Chanping
                INSERT INTO Chanping (ChanpingName, Danjia) VALUES
                ('Laptop', 1200.00),
                ('Mouse', 25.50),
                ('Keyboard', 49.99),
                ('Monitor', 350.00);

                -- Insert multiple records into Dingdan
                INSERT INTO Dingdan (DingdanRiqi, GukeID) VALUES
                ('2025-12-01', 101),
                ('2025-12-05', 102),
                ('2025-12-10', 101);

                -- Insert multiple records into DingdanMingxi
                INSERT INTO DingdanMingxi (DingdanID, ChanpingID, Shuliang) VALUES
                (1, 1, 1), -- Order 1: 1 Laptop
                (1, 2, 2), -- Order 1: 2 Mouses
                (2, 3, 1), -- Order 2: 1 Keyboard
                (3, 1, 1), -- Order 3: 1 Laptop
                (3, 4, 2); -- Order 3: 2 Monitors

    '''
    # Execute the entire script at once
    cursor.executescript(sql_txt_create)    
    cursor.close()

    #query db to get data into dataframe
    #query to join mutliple table to get summary
    sql_txt_query  = '''
            SELECT
            O.DingdanID,
            O.DingdanRiqi,
            P.ChanpingName,
            OD.Shuliang,
            P.Danjia,
            (OD.Shuliang * P.Danjia) AS TotalPerItem
            FROM
                Dingdan O
            INNER JOIN
                DingdanMingxi OD ON O.DingdanID = OD.DingdanID
            INNER JOIN
                Chanping P ON OD.ChanpingID = P.ChanpingID
            ORDER BY
                O.DingdanID, P.ChanpingName;

    '''
    #cursor.execute(sql_txt_query)
    df = pd.read_sql_query(sql_txt_query, conn)
df




,DingdanID,DingdanRiqi,ChanpingName,Shuliang,Danjia,TotalPerItem
0,1,2025-12-01,Laptop,1,1200.00,1200.00
1,1,2025-12-01,Mouse,2,25.50,51.00
2,2,2025-12-05,Keyboard,1,49.99,49.99
3,3,2025-12-10,Laptop,1,1200.00,1200.00
4,3,2025-12-10,Monitor,2,350.00,700.00


In [16]:
#https://docs.langchain.com/oss/python/langchain/sql-agent
from langchain_community.utilities import SQLDatabase
db2 = SQLDatabase.from_uri(r"sqlite:///D:/data science data/ai_sqlite2.db")
print(f"Dialect: {db2.dialect}")
print(f"Available tables: {db2.get_usable_table_names()}")
print(f'Sample output: {db2.run("SELECT * FROM Chanping LIMIT 5;")}')

Dialect: sqlite
Available tables: ['Chanping', 'Dingdan', 'DingdanMingxi']
Sample output: [(1, 'Laptop', 1200.0), (2, 'Mouse', 25.5), (3, 'Keyboard', 49.99), (4, 'Monitor', 350.0)]


In [17]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit2 = SQLDatabaseToolkit(db=db2, llm=llm)

tools2 = toolkit2.get_tools()

for tool2 in tools2:
    print(f"{tool2.name}: {tool2.description}\n")

sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



In [18]:
#Use create_agent
system_prompt2 = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
    dialect=db2.dialect,
    top_k=10,
)

In [19]:
from langchain.agents import create_agent
agent2 = create_agent(
    llm,
    tools2,
    system_prompt=system_prompt2,
)

In [20]:
question = "what product is most expensive?"

for step in agent2.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

what product is most expensive?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (038138eb-6c45-4969-8e9f-7f5fa3228d6b)
 Call ID: 038138eb-6c45-4969-8e9f-7f5fa3228d6b
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Chanping, Dingdan, DingdanMingxi


ChatGoogleGenerativeAIError: Error calling model 'gemini-2.5-flash' (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 31.294804745s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerM

In [21]:
question = "what ChanpingName is most expensive?"

for step in agent2.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

what ChanpingName is most expensive?


ChatGoogleGenerativeAIError: Error calling model 'gemini-2.5-flash' (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 45.001696257s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerM